In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_dir = '/content/drive/MyDrive/10 monkey/training/training'
test_dir = '/content/drive/MyDrive/10 monkey/validation/validation'

In [3]:
cols = ['Label', 'Latin Name', 'Common Name', 'Train Images', 'Validation Images']
df = pd.read_csv('/content/drive/MyDrive/10 monkey/monkey_labels.txt', names=cols, skiprows=1)

In [4]:
df.head()

,Label,Latin Name,Common Name,Train Images,Validation Images
0,n0,alouatta_palliata\t,mantled_howler,131,26
1,n1,erythrocebus_patas\t,patas_monkey,139,28
2,n2,cacajao_calvus\t,bald_uakari,137,27
3,n3,macaca_fuscata\t,japanese_macaque,152,30
4,n4,cebuella_pygmea\t,pygmy_marmoset,131,26


In [6]:

from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential

In [7]:
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

In [8]:
IMAGE_SIZE = [100, 100]

In [16]:
from keras.preprocessing.image import ImageDataGenerator

In [17]:

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [18]:
training_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size = (100, 100),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1145 images belonging to 10 classes.


In [19]:

test_set = test_datagen.flow_from_directory(test_dir,
                                            target_size = (100, 100),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 272 images belonging to 10 classes.


In [23]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

9412608/9406464 [==============================] - 0s 0us/step


In [24]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [26]:
history = model.fit(
    training_set,
    validation_data=test_set,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/50
36/36 [==============================] - 678s 19s/step - loss: 2.0256 - accuracy: 0.3305 - val_loss: 0.8466 - val_accuracy: 0.7610
Epoch 2/50
36/36 [==============================] - 48s 1s/step - loss: 0.7669 - accuracy: 0.7279 - val_loss: 0.8006 - val_accuracy: 0.7426
Epoch 3/50
36/36 [==============================] - 48s 1s/step - loss: 0.5789 - accuracy: 0.8047 - val_loss: 0.6094 - val_accuracy: 0.8162
Epoch 4/50
36/36 [==============================] - 48s 1s/step - loss: 0.4906 - accuracy: 0.8391 - val_loss: 0.6322 - val_accuracy: 0.7904
Epoch 5/50
36/36 [==============================] - 48s 1s/step - loss: 0.5086 - accuracy: 0.8267 - val_loss: 0.7559 - val_accuracy: 0.7757
Epoch 6/50
36/36 [==============================] - 48s 1s/step - loss: 0.4404 - accuracy: 0.8521 - val_loss: 0.7084 - val_accuracy: 0.7794


In [27]:
import tensorflow as tf

from keras.models import load_model

model.save('mobilenet.h5')